In [1]:
import pandas as pd
import statsmodels.api as sm

In [16]:
from statsmodels.stats.power import TTestIndPower


effect_size = 0.39455 / 0.003  
alpha = 0.05  
nobs = 23659  

# Calculate power
power_analysis = TTestIndPower()
power = power_analysis.solve_power(effect_size=effect_size, nobs1=nobs, alpha=alpha, ratio=.01)

power

1.0

In [2]:
game_details = pd.read_csv('games_details.csv')
game_details

/srv/conda/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


GAME_ID     TEAM_ID TEAM_ABBREVIATION    TEAM_CITY  PLAYER_ID  \
0       22200477  1610612759               SAS  San Antonio    1629641   
1       22200477  1610612759               SAS  San Antonio    1631110   
2       22200477  1610612759               SAS  San Antonio    1627751   
3       22200477  1610612759               SAS  San Antonio    1630170   
4       22200477  1610612759               SAS  San Antonio    1630200   
...          ...         ...               ...          ...        ...   
668623  11200005  1610612743               DEN       Denver     202706   
668624  11200005  1610612743               DEN       Denver     202702   
668625  11200005  1610612743               DEN       Denver     201585   
668626  11200005  1610612743               DEN       Denver     202389   
668627  11200005  1610612743               DEN       Denver     201951   

            PLAYER_NAME NICKNAME START_POSITION COMMENT    MIN  ...  OREB  \
0        Romeo Langford    Romeo              F     NaN  18:06  ...   1.0   
1         Jeremy Sochan   Jeremy              F     NaN  31:01  ...   6.0   
2          Jakob Poeltl    Jakob              C     NaN  21:42  ...   1.0   
3         Devin Vassell    Devin              G     NaN  30:20  ...   0.0   
4             Tre Jones      Tre              G     NaN  27:44  ...   0.0   
...                 ...      ...            ...     ...    ...  ...   ...   
668623  Jordan Hamilton      NaN            NaN     NaN     19  ...   0.0   
668624   Kenneth Faried      NaN            NaN     NaN     23  ...   1.0   
668625     Kosta Koufos      NaN            NaN     NaN     15  ...   3.0   
668626   Timofey Mozgov      NaN            NaN     NaN     19  ...   1.0   
668627        Ty Lawson      NaN            NaN     NaN     27  ...   0.0   

        DREB  REB  AST  STL  BLK   TO   PF   PTS  PLUS_MINUS  
0        1.0  2.0  0.0  1.0  0.0  2.0  5.0   2.0        -2.0  
1        3.0  9.0  6.0  1.0  0.0  2.0  1.0  23.0       -14.0  
2        3.0  4.0  1.0  1.0  0.0  2.0  4.0  13.0        -4.0  
3        9.0  9.0  5.0  3.0  0.0  2.0  1.0  10.0       -18.0  
4        2.0  2.0  3.0  0.0  0.0  2.0  2.0  19.0         0.0  
...      ...  ...  ...  ...  ...  ...  ...   ...         ...  
668623   2.0  2.0  0.0  2.0  0.0  1.0  3.0  17.0         NaN  
668624   0.0  1.0  1.0  1.0  0.0  3.0  3.0  18.0         NaN  
668625   5.0  8.0  0.0  1.0  0.0  0.0  3.0   6.0         NaN  
668626   2.0  3.0  1.0  0.0  0.0  4.0  2.0   2.0         NaN  
668627   2.0  2.0  6.0  2.0  0.0  6.0  1.0   8.0         NaN  

[668628 rows x 29 columns]

In [3]:
nan_count_min = game_details['COMMENT'].isna().sum()

nan_count_min


558939

In [4]:
game_details['MIN'] = pd.to_numeric(game_details['MIN'], errors='coerce')
game_details['REB'] = pd.to_numeric(game_details['REB'], errors='coerce')
game_details['AST'] = pd.to_numeric(game_details['AST'], errors='coerce')
game_details['STL'] = pd.to_numeric(game_details['STL'], errors='coerce')
game_details['BLK'] = pd.to_numeric(game_details['BLK'], errors='coerce')
game_details['TO'] = pd.to_numeric(game_details['TO'], errors='coerce')
game_details['PTS'] = pd.to_numeric(game_details['PTS'], errors='coerce')

grouped_game_details = game_details.groupby('PLAYER_NAME').agg({
    'MIN': 'sum',  
    'REB': 'sum',  
    'AST': 'sum',  
    'STL': 'sum',  
    'BLK': 'sum', 
    'PTS': 'sum',
    'TO': 'sum'    
}).reset_index()


grouped_game_details = grouped_game_details[grouped_game_details['MIN'] > 500]


grouped_game_details['PER'] = (grouped_game_details['REB'] + grouped_game_details['AST'] +
                               grouped_game_details['STL'] + grouped_game_details['PTS'] + 
                               grouped_game_details['BLK'] - grouped_game_details['TO']) / grouped_game_details['MIN']


sorted_players = grouped_game_details.sort_values(by='PER', ascending=False)

# Select the top 100 players
top_100_players = sorted_players.head(100)
top_100_players

PLAYER_NAME     MIN      REB      AST     STL     BLK      PTS  \
2207  Russell Westbrook   639.0   8754.0  10045.0  1993.0   375.0  27180.0   
1095       James Harden   670.0   6497.0   7848.0  1815.0   618.0  28460.0   
1593       LeBron James  1237.0  13147.0  12472.0  2726.0  1346.0  46638.0   
2340      Stephen Curry   736.0   4997.0   6761.0  1728.0   259.0  25826.0   
2023        Paul George   623.0   5969.0   3313.0  1549.0   412.0  18648.0   
...                 ...     ...      ...      ...     ...     ...      ...   
97       Andre Iguodala  1631.0   7178.0   6042.0  2103.0   749.0  16422.0   
1020       JaVale McGee   763.0   5058.0    427.0   388.0  1394.0   7675.0   
1547        Kyle Korver  1134.0   4224.0   2408.0   933.0   484.0  13925.0   
1369         Josh Smith  1453.0   7550.0   3154.0  1230.0  1901.0  14855.0   
233         Brad Miller   767.0   4565.0   2124.0   484.0   465.0   7274.0   

          TO        PER  
2207  4938.0  67.932707  
1095  4345.0  61.034328  
1593  6038.0  56.823767  
2340  3341.0  49.225543  
2023  2532.0  43.914928  
...      ...        ...  
97    2653.0  18.296137  
1020  1015.0  18.252949  
1547  1530.0  18.028219  
1369  2530.0  18.004129  
233   1118.0  17.984355  

[100 rows x 9 columns]

In [1]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.stats.multitest import multipletests


X = sm.add_constant(top_100_players['MIN'])
y = top_100_players['PER']


model = sm.OLS(y, X).fit()
p_values = model.pvalues

bonferroni_results = multipletests(p_values, alpha=0.05, method='bonferroni')

bh_results = multipletests(p_values, alpha=0.05, method='fdr_bh')

summary_df = pd.DataFrame({
    'coef': model.params,
    'std_err': model.bse,
    't': model.tvalues,
    'p_value': model.pvalues,
    'bonferroni_p_value': bonferroni_results[1],
    'bh_p_value': bh_results[1],
    'bonferroni_reject': bonferroni_results[0],
    'bh_reject': bh_results[0]
})

bonferroni_discoveries = summary_df['bonferroni_reject'].sum()
bh_discoveries = summary_df['bh_reject'].sum()


print(f"Discoveries with FWER control (Bonferroni): {bonferroni_discoveries}")
print(f"Discoveries with FDR control (Benjamini-Hochberg): {bh_discoveries}")
print(model.summary())

NameError: name 'top_100_players' is not defined

In [2]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.stats.multitest import multipletests


X = sm.add_constant(top_100_players[['MIN', 'TO']])

y = top_100_players['PER']

model = sm.OLS(y, X).fit()

p_values = model.pvalues

bonferroni_results = multipletests(p_values, alpha=0.05, method='bonferroni')

bh_results = multipletests(p_values, alpha=0.05, method='fdr_bh')

bonferroni_discoveries = bonferroni_results[0].sum()
bh_discoveries = bh_results[0].sum()

print(f"Discoveries with FWER control (Bonferroni): {bonferroni_discoveries}")
print(f"Discoveries with FDR control (Benjamini-Hochberg): {bh_discoveries}")
print(model.summary())

NameError: name 'top_100_players' is not defined

In [7]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.stats.multitest import multipletests

game_details = game_details.dropna(subset=['MIN'])

game_details['MIN'] = pd.to_numeric(game_details['MIN'], errors='coerce')

game_details = game_details.dropna(subset=['MIN'])

game_details['FG_PCT'] = game_details['FG_PCT'].astype(float)

X = sm.add_constant(game_details['MIN'])
y = game_details['FG_PCT']

model = sm.OLS(y, X).fit()

p_values = model.pvalues

bonferroni_results = multipletests(p_values, alpha=0.05, method='bonferroni')

bh_results = multipletests(p_values, alpha=0.05, method='fdr_bh')

summary_df = pd.DataFrame({
    'coef': model.params,
    'std_err': model.bse,
    't': model.tvalues,
    'p_value': model.pvalues,
    'bonferroni_p_value': bonferroni_results[1],
    'bh_p_value': bh_results[1],
    'bonferroni_reject': bonferroni_results[0],
    'bh_reject': bh_results[0]
})

bonferroni_discoveries = summary_df['bonferroni_reject'].sum()
bh_discoveries = summary_df['bh_reject'].sum()
summary = model.summary()

print(summary)
print(f"Discoveries with FWER control (Bonferroni): {bonferroni_discoveries}")
print(f"Discoveries with FDR control (Benjamini-Hochberg): {bh_discoveries}")

                            OLS Regression Results                            
Dep. Variable:                 FG_PCT   R-squared:                       0.038
Model:                            OLS   Adj. R-squared:                  0.038
Method:                 Least Squares   F-statistic:                     941.0
Date:                Mon, 11 Dec 2023   Prob (F-statistic):          1.12e-202
Time:                        18:26:50   Log-Likelihood:                -1309.4
No. Observations:               23661   AIC:                             2623.
Df Residuals:                   23659   BIC:                             2639.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.3031      0.004     80.956      0.0

/tmp/ipykernel_1823/751845214.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  game_details['MIN'] = pd.to_numeric(game_details['MIN'], errors='coerce')
